In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

import achiralqw as aqw
import scipy.sparse as sparse

from achiralqw.graph import QWGraph, QWGraphBuilder as qwgb
from achiralqw.plotter import plot_qwgraph
from achiralqw.collection import CachedQWGraphCollection

from achiralqw.article import * 

In [ ]:
N= 20

smat = sparse.dok_matrix((N,N), dtype = "complex")
ring = sparse.identity(N, dtype = "complex", format = "dok")*4

In [ ]:
print(smat[0,0])
print(ring[3,3])

ring.shape[1]

In [ ]:
for i in range(N):
    ring[i,(i+1)%N] = complex(-1)
    ring[(i+1)%N,i] = complex(-1)

In [ ]:
print(ring.get_shape())
print(ring.todense())

In [ ]:
sval, svec = sparse.linalg.eigsh(ring, k = N-2)
val, vec = np.linalg.eigh(ring.todense())

print(val)
print(sval)

np.reshape(sval, (N-2,1))

In [ ]:
def to_adjacency(mat):
    
    out = sparse.dok_matrix(mat.shape, dtype = "int")
    
    for k in mat.keys():
        if k[0] != k[1]:
            out[k] = 1
            
    return out
        
mat = to_adjacency(ring)
nx.draw_kamada_kawai(nx.from_numpy_matrix(mat.todense()))
plt.show()


mst = sparse.csgraph.minimum_spanning_tree(ring).todok()
print(mst.todense())
nx.draw_circular(nx.from_numpy_matrix(mst.todense()))
plt.show()
for edge in ring.keys():
    if abs(mst[edge]) > 1e-6 :
        print(edge)
        
        
        


In [ ]:
def sparse_mst( smat ):
    #do it yourself implementation of kruskal algorithm for a 01 adjacency matrix
    #suited for dok matrices
    
    #union find parent list
    par = np.arange(0,smat.shape[0])
    
    out = sparse.dok_matrix(smat.shape, dtype = "int")
    
    #primitives for union-find
    def same(a,b) ->bool :
        
        p1 = par[a]
        while p1 != par[p1] :
            p1 = par[p1]
            par[a] = p1
        
        p2 = par[b]
        while p2 != par[p2] :
            p2 = par[p2]
            par[b] = p2
            
        #print( "same {} {}\t".format(a,b), par)
        return p1 == p2
        
    def join(a,b) -> None :
        
        p1 = par[a]
        while p1 != par[p1] :
            p1 = par[p1]
            par[a] = p1
        
        p2 = par[b]
        while p2 != par[p2] :
            p2 = par[p2]
            par[b] = p2
            
        par[p2] = p1
        
    for edge in smat.keys():
        if not same(edge[0], edge[1]):
            join(edge[0], edge[1])
            out[edge] = 1
            out[edge[1],edge[0]] = 1
            
    return out

my_mst = sparse_mst(ring)
print(my_mst.todense())
nx.draw_circular(nx.from_numpy_matrix(my_mst.todense()))
plt.show()

for edge in ring.keys():
    if edge[0] > edge[1] and my_mst[edge] == 0 :
        print(edge)
    

In [ ]:
a = my_mst

sparse.csgraph.shortest_path(a, method = "D", directed = False, unweighted= True, indices = 3)

In [ ]:
target = 6

print( ring[:,[0,7]])
print( ring.todense()[:,[0,7]])
    
ring[:,[0,7]] = ring[:,[7, 0]]

print( ring[:,[0,7]])
print( ring.todense()[:,[0,7]])

print(ring.todense())